use the titanic dataset and split into train and test the same way we did in hw3

fit a neural network using independent variables pclass sex age and sibsp
dependent variable survived

omit all na examples

use 2 hidden layers and set the activation functions to be sigmoid funciton 
use the adam optimizer

create a confusion matrix for classification 

compare the in sample and out of sample accuracy with the pruned decision tree obtained in hw 3  use a table

use skelarn

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
datapath: str = os.path.join('data', 'Titanic.csv')

In [3]:
df: pd.DataFrame = pd.read_csv(datapath)

In [4]:
df.head()

,Unnamed: 0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [5]:
df = df.drop(["Unnamed: 0", "name", "parch", "fare", "body", "embarked", "boat", "home.dest", "cabin", "ticket"], axis=1)

In [6]:
df.isna().sum()

pclass        0
survived      0
sex           0
age         263
sibsp         0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.isna().sum()

pclass      0
survived    0
sex         0
age         0
sibsp       0
dtype: int64

In [9]:
df['sex'] = df['sex'].astype('category')
sex_categories = df['sex'].cat.categories.tolist()
df['pclass'] = df['pclass'].astype('category')
pclass_categories = df['pclass'].cat.categories.tolist()
df['sex'] = df['sex'].cat.codes
df['pclass'] = df['pclass'].cat.codes

In [10]:
def prepare(df, target, val_percent, test_percent, shuffle=True):
    assert 0 < test_percent < 1.
    
    num_test = round(len(df) * test_percent)
    y = df.loc[:, df.columns == target]
    x = df.loc[:, df.columns != target]
    
    X, x_test, y, y_test = train_test_split(x, y, test_size=test_percent, shuffle=shuffle)
        
    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=val_percent)
    
    return x_train, y_train, x_val, y_val, x_test, y_test

In [11]:
x_train, y_train, x_val, y_val, x_test, y_test = prepare(df, 'survived', 0.2, 0.2, shuffle=True)

In [12]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(4, 8), random_state=42)

In [13]:
df.head()

,pclass,survived,sex,age,sibsp
0,0,1,0,29.0000,0
1,0,1,1,0.9167,1
2,0,0,0,2.0000,1
3,0,0,1,30.0000,1
4,0,0,0,25.0000,1


In [14]:
clf.fit(x_train, y_train)

/home/mycicle/anaconda3/envs/cs583/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/mycicle/anaconda3/envs/cs583/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 8), random_state=42)

### In sample accuracy

In [15]:
y_pred = clf.predict(x_train)
print(classification_report(y_pred, y_train))
print(clf.score(x_train, y_train))

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       437
           1       0.68      0.78      0.73       231

    accuracy                           0.79       668
   macro avg       0.78      0.79      0.78       668
weighted avg       0.81      0.79      0.80       668

0.7949101796407185


### Out of sample accuracy

In [16]:
y_pred_test = clf.predict(x_test)
print(classification_report(y_pred_test, y_test))
print(clf.score(x_test, y_test))

              precision    recall  f1-score   support

           0       0.88      0.79      0.84       135
           1       0.69      0.81      0.74        75

    accuracy                           0.80       210
   macro avg       0.78      0.80      0.79       210
weighted avg       0.81      0.80      0.80       210

0.8


In [19]:
in_sample = pd.DataFrame({'mlp': [clf.score(x_train, y_train)], 'dt': [0.77]})
out_of_sample = pd.DataFrame({'mlp': [clf.score(x_test, y_test)], 'dt': [0.79]})

## In sample classification accuracy comparison with decision tree

In [20]:
in_sample

,mlp,dt
0,0.79491,0.77


## Out of sample classification accuracy comparison with decision tree

In [21]:
out_of_sample

,mlp,dt
0,0.8,0.79
